<a href="https://colab.research.google.com/github/alyrazik/CUDA/blob/main/CUDA_code_coursework_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get --purge remove cuda nvidia* libnvidia-*
!dpkg -l | grep cuda- | awk '{print $2}' | xargs -n1 dpkg --purge
!apt-get remove cuda-*
!apt autoremove
!apt-get update

In [ ]:
!wget https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64 -O cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!dpkg -i cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!apt-key add /var/cuda-repo-9-2-local/7fa2af80.pub
!apt-get update
!apt-get install cuda-9.2

In [ ]:
!nvcc --version

In [4]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git

  Cloning git://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-gv2f49xb
  Running command git clone -q git://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-gv2f49xb
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-cp37-none-any.whl size=4307 sha256=4b28d51fd03664de5fcf91e1decfe4b2520061d4f1b7eb4fef2df20726724a43
  Stored in directory: /tmp/pip-ephem-wheel-cache-craxws4s/wheels/10/c2/05/ca241da37bff77d60d31a9174f988109c61ba989e4d4650516
Successfully built NVCCPlugin


In [5]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


In [ ]:
 %reload_ext nvcc_plugin

#HERE IT IS... THE FINAL PIECE OF CODE

In [27]:
%%cu 
#include <iostream> 
#include <stdlib.h>
#include <cstdio>
#include <cublas_v2.h>
#include <cuda_runtime.h>
#include <chrono>
using namespace std; 

//#define N 10
//#define M 10


// __global__ means this function will be executed on the GPU and is callable from the host.
// this version of the function parallizes the outer loop only.

__global__ void GTensorsOperation(float* a, float* b, float* c, float* d, int n, int m)
{

  int i = threadIdx.x; // i is unique for each thread.
 
if (i<n) 
// so we won't run into a problem in case the number of threads exceeded the...
// array elements N. (we'll think of a better way to parllelize the second loop
// as well and them make SIZE n*m
   for (int j=0; j<m; j++) 
   {
       d[i*m+j]=a[i]*b[j]+c[i*m+j];
   }
      
} 

// __global__ means this function will be executed on the GPU and is callable from the host.
// this version of the function parallizes the inner loop as well as the outer loop.

__global__ void G2TensorsOperation(float* a, float* b, float* c, float* d, int n, int m)
{

  int i = threadIdx.x; // i is unique for each thread.
 
if (i<n*m) 
// so we won't run into a problem in case the number of threads exceeded the...
// array elements N*M. 
      //i/M counts one (increments) every M elements until it reaches the count of N.
      //i%M counts M times (from 0 to M-1) and then repeats this series N times.
      //this basically accomplishes the multiplication process between the two vectors
      d[i]=a[i/m]*b[i%m]+c[i]; 
}
// and the CPU implementation.

void TensorsOperation(float* a, float* b, float* c, float* d, int n, int m)
{
 for (int i=0; i<n; i++)
   for (int j=0; j<m; j++) 
   {
       d[i*m+j]=a[i]*b[j]+c[i*m+j];
   }
      
}




int main() 
{ 
    
// Generate a, b, c matrices with random elements using N and M
// they will be used to initalize the variables for both CPU and GPU.
cout<<"N       M       GPU1        GPU(NxM)       CPU"<<endl;
int N;
int M=10;
for (N =10; N<=10000; N=10*N)
{
//cout<<"\nLooping with M="<<M<<" and N="<<N<<endl;
cout<<N<<"\t"<<M<<"\t";

float* A = new float[N];
float* B = new float[M];
float* C = new float[N*M];

for(int i=0;i<N;i++)
      A[i]=rand()%100;
for(int i=0;i<M;i++)
      B[i]=rand()%100;
for(int i=0;i<N*M;i++)
      C[i]=rand()%100;


//cout << "This code runs the operation on GPU\n";
int n=N;
int m=M;

float*a, *b, *c, *d;
//allocate memory that is shared between the GPU device and the host CPU.
cudaMallocManaged(&a, n*sizeof(float)); 
cudaMallocManaged(&b, m*sizeof(float));
cudaMallocManaged(&c, n*m*sizeof(float));
cudaMallocManaged(&d, n*m*sizeof(float));

//initialize matrices
for(int i=0;i<N;i++)
      a[i]=A[i];
for(int i=0;i<M;i++)
      b[i]=B[i];
for(int i=0;i<N*M;i++)
      c[i]=C[i];

//a[0]=1; a[1]=2; a[2]=3;
//b[0]=4; b[1]=2;
//c[0]=2; c[1]=1; c[2]=1; c[3]=2; c[4]=4; c[5]=5;


//to calculate the time
cudaEvent_t start, end;
cudaEventCreate(&start); 
cudaEventCreate(&end); 
cudaEventRecord(start);

//Kernel launch
 
GTensorsOperation<<<1, N>>>(a, b, c, d, n, m); 

//1 is no. of thread blocks
//3, the second p/m is the number of threads within each block. usually the number of elements in a vector, the number of iterations.

cudaEventRecord(end); 
//get the CPU to wait for all kernels to finish.
//cudaDeviceSynchronize(); //do I need it?
cudaEventSynchronize(end);  

float time = 0; 
cudaEventElapsedTime(&time, start, end);  
//cout<<"Time taken "
cout<<time<<"    ";
/*
cout<<"Result matrix D: "<<endl;
for (int i =0; i<n; i++)
 {
  for (int j = 0; j<m; j++)
    cout<<d[i*m+j]<<" ";
  cout<<endl;
 }
*/

//Try the next function  G2TensorsOperation<<<1, N>>>(a, b, c, d, n, m); 
 
//cout << "\nThis code runs the operation on GPU using N*M threads.\n";
cudaEventRecord(start);
G2TensorsOperation<<<1, N*M>>>(a, b, c, d, n, m); 
cudaEventRecord(end); 
cudaEventSynchronize(end); 
time = 0;
cudaEventElapsedTime(&time, start, end);  
// cudaEventElapsedTime computes elapsed time between two events (in milliseconds with a resolution of around 0.5 microseconds)
// source: docs.nvidia.com
//cout<<"Time taken "<<
cout<<time<<"    ";
 
/*
cout<<"Result matrix D: "<<endl;
for (int i =0; i<n; i++)
 {
  for (int j = 0; j<m; j++)
    cout<<d[i*m+j]<<" ";
  cout<<endl;
 }
*/



//free the allocated memory
cudaFree(a);
cudaFree(b);
cudaFree(c);
cudaFree(d);


//now the CPU version:

//cout << "\nThis code runs the operation on CPU\n";
float* a1 = new float[n];
float* b1 = new float[m];
float* c1 = new float[n*m];
float* d1 = new float[n*m];

//initialize matrices
for(int i=0;i<N;i++)
      a1[i]=A[i];
for(int i=0;i<M;i++)
      b1[i]=B[i];
for(int i=0;i<N*M;i++)
      c1[i]=C[i];

//calculate time
using chrono::high_resolution_clock;
using chrono::duration_cast;
using chrono::duration;
using chrono::milliseconds;
auto t_start = high_resolution_clock::now();

 TensorsOperation(a1, b1, c1, d1, n, m);
auto t_end = high_resolution_clock::now();

duration<double, std::milli> ms_double = t_end - t_start;
//cout << "Time taken: " 
cout<< ms_double.count() <<endl;

/*
cout<<"Result matrix D: \n" ;
for (int i =0; i<n; i++)
 {
  for (int j = 0; j<m; j++)
    cout<<d1[i*m+j]<<" ";
  cout<<endl;
 }
*/
//cout<<endl;
M=M*10; 


}
return 0; 
} 


N       M       GPU1        GPU(NxM)       CPU
10	10	0.440704    0.012256    0.000713
100	100	0.1992    0.002048    0.035488
1000	1000	7.34189    0.002752    3.6769
10000	10000	0.005472    0.002496    501.149

